# 📘 Agentic Architectures 5: Multi-Agent Systems

In this notebook, we advance to one of the most powerful and flexible architectures: the **Multi-Agent System**. This pattern moves beyond the concept of a single agent, no matter how complex, and instead models a team of specialized agents that collaborate to solve a problem. Each agent has a distinct role, persona, and set of skills, mirroring how human expert teams work.

This approach allows for a profound 'division of labor', where complex problems are broken down into sub-tasks and assigned to the agent best suited for the job. To showcase its power, we will conduct a direct comparison. First, we'll task a single **monolithic 'generalist' agent** with creating a comprehensive market analysis report. Then, we will assemble a **specialist team**—a Technical Analyst, a News Analyst, and a Financial Analyst—and have a fourth 'Manager' agent synthesize their expert inputs into a final report. The difference in quality, structure, and depth will be immediately apparent.

### Definition
A **Multi-Agent System** is an architecture where a group of distinct, specialized agents collaborate (or sometimes compete) to achieve a common goal. A central controller or a defined workflow protocol is used to manage communication and route tasks between the agents.

### High-level Workflow

1.  **Decomposition:** A main controller or the user provides a complex task.
2.  **Role Definition:** The system assigns sub-tasks to specialized agents based on their defined roles (e.g., 'Researcher', 'Coder', 'Critic', 'Writer').
3.  **Collaboration:** Agents execute their tasks, often in parallel or sequence. They pass their outputs to each other or to a central 'blackboard'.
4.  **Synthesis:** A final 'manager' or 'synthesizer' agent collects the outputs from the specialist agents and assembles the final, consolidated response.

### When to Use / Applications
*   **Complex Report Generation:** Creating detailed reports that require expertise from multiple domains (e.g., financial analysis, scientific research).
*   **Software Development Pipelines:** Simulating a dev team with a programmer, a code reviewer, a tester, and a project manager.
*   **Creative Brainstorming:** A team of agents with different 'personalities' (e.g., one optimistic, one cautious, one wildly creative) can generate a more diverse set of ideas.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Specialization & Depth:** Each agent can be fine-tuned with a specific persona and tools, leading to higher-quality work in its domain.
    *   **Modularity & Scalability:** It's easy to add, remove, or upgrade individual agents without redesigning the entire system.
    *   **Parallelism:** Multiple agents can work on their sub-tasks simultaneously, potentially reducing overall task time.
*   **Weaknesses:**
    *   **Coordination Overhead:** Managing the communication and workflow between agents adds complexity to the system design.
    *   **Increased Cost & Latency:** Running multiple agents involves more LLM calls, which can be more expensive and slower than a single-agent approach.

## Phase 0: Foundation & Setup

We will begin by installing our libraries and configuring our API keys for Together, LangSmith, and Tavily.

### Step 0.1: Installing Core Libraries

**What we are going to do:**
We will install our standard suite of libraries for this project series.

In [ ]:
#!pip install -q -U langchain-together langchain langgraph rich python-dotenv langchain-tavily

### Step 0.2: Importing Libraries and Setting Up Keys

**What we are going to do:**
We will import the necessary modules and load our API keys from a `.env` file.

In [ ]:
import os
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_together import ChatTogether
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "05_TIDIT_Workshop"

for key in ["TOGETHER_API_KEY", "LANGSMITH_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

## Phase 1: The Baseline - A Monolithic 'Generalist' Agent

To showcase the value of a specialist team, we first need to see how a single agent performs on a complex task. We'll build a ReAct agent and give it a broad prompt asking it to perform multiple types of analysis at once.

### Step 1.1: Building the Monolithic Agent

**What we are going to do:**
We will construct a standard ReAct agent. We'll provide it with a web search tool and a very general system prompt that asks it to be a comprehensive financial analyst.

In [ ]:
console = Console()

# Define the shared state for both agents
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Define the tool and LLM
search_tool = TavilySearch(max_results=3, tavily_api_key=os.environ["TAVILY_API_KEY"])
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0
)
# All specialists share this single tool-binding. This is intentional: every agent
# in this demo uses the same search tool. If specialists needed different tool sets,
# each would get its own bind_tools() call with a distinct tool list.
llm_with_tools = llm.bind_tools([search_tool])

# Define the monolithic agent node
def monolithic_agent_node(state: AgentState):
    console.print("--- MONOLITHIC AGENT: Thinking... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tool_node = ToolNode([search_tool])

# Build the ReAct graph for the monolithic agent
mono_graph_builder = StateGraph(AgentState)
mono_graph_builder.add_node("agent", monolithic_agent_node)
mono_graph_builder.add_node("tools", tool_node)
mono_graph_builder.set_entry_point("agent")

mono_graph_builder.add_conditional_edges("agent", tools_condition)
mono_graph_builder.add_edge("tools", "agent")

monolithic_agent_app = mono_graph_builder.compile()


def get_last_ai_content(messages):
    """Get the content of the last AIMessage with non-empty content (handles tool-call-only final turns and list content)."""
    for m in reversed(messages):
        if not isinstance(m, AIMessage):
            continue
        c = getattr(m, "content", None)
        if c is None:
            continue
        if isinstance(c, str) and c.strip():
            return c
        if isinstance(c, list):
            parts = []
            for block in c:
                if isinstance(block, str):
                    parts.append(block)
                elif isinstance(block, dict) and "text" in block:
                    parts.append(block["text"])
            text = "".join(parts).strip()
            if text:
                return text
    return "(No text response from agent.)"


print("Monolithic 'generalist' agent compiled successfully.")

### Step 1.2: Testing the Monolithic Agent

**What we are going to do:**
We'll give the generalist agent a complex task: create a full market analysis report for a company, covering three distinct areas.

In [ ]:
company = "NVIDIA (NVDA)"
monolithic_query = f"Create a brief but comprehensive market analysis report for {company}. The report should include three sections: 1. A summary of recent news and market sentiment. 2. A basic technical analysis of the stock's price trend. 3. A look at the company's recent financial performance."

console.print(f"[bold yellow]Testing MONOLITHIC agent on a multi-faceted task:[/bold yellow]\n'{monolithic_query}'\n")

final_mono_output = monolithic_agent_app.invoke(
    {
        "messages": [
            SystemMessage(content="You are a single, expert financial analyst. You must create a comprehensive report covering all aspects of the user's request. Use the search tool to gather information, then provide your final report in one message. Do not end with only tool calls—always conclude with your full written report."),
            HumanMessage(content=monolithic_query)
        ]
    },
    config={"recursion_limit": 50},
)

console.print("\n--- [bold red]Final Report from Monolithic Agent[/bold red] ---")
console.print(Markdown(get_last_ai_content(final_mono_output['messages'])))

**Discussion of the Output:**
The monolithic agent produced a report. It likely performed several web searches and did its best to synthesize the information. However, the output may have some weaknesses:
- **Lack of Structure:** The sections might blend together, without clear headings or a professional format.
- **Superficial Analysis:** Trying to be an expert in three domains at once, the agent might provide only high-level summaries without much depth in any single area.
- **Generic Tone:** The language might be generic, lacking the specific jargon and focus of a true specialist in each field.

This result is our baseline. It's functional, but not exceptional. Now, we will build a specialist team to see if we can do better.

## Phase 2: The Advanced Approach - A Multi-Agent Specialist Team

Now we'll build our team: a News Analyst, a Technical Analyst, and a Financial Analyst. Each will be its own agent node with a specific persona. A final Report Writer will act as the manager, compiling their work.

### Step 2.1: Defining the Specialist Agent Nodes

**What we are going to do:**
We will create three distinct agent nodes. The key difference is the highly specific system prompt we give each one. This prompt defines their persona, their area of expertise, and the exact format their output should take. This is how we enforce specialization.

In [ ]:
# The state for our multi-agent system will hold the outputs of each specialist
class MultiAgentState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

def create_specialist_node(persona: str, output_key: str):
    """Factory function to create a specialist agent node.

    Each specialist gets its own mini ReAct sub-graph so that tool calls
    (e.g. Tavily web search) are actually executed and the results fed
    back to the LLM before it writes its report section.
    """
    system_prompt = (
        persona
        + "\n\nYou have access to a web search tool. Use it to gather real, "
        "up-to-date data. Then provide your final report section formatted "
        "in markdown, focusing only on your area of expertise. Do not end "
        "with only tool calls—always conclude with your full written report."
    )

    def _specialist_agent(state: AgentState):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    sub_graph = StateGraph(AgentState)
    sub_graph.add_node("agent", _specialist_agent)
    sub_graph.add_node("tools", ToolNode([search_tool]))
    sub_graph.set_entry_point("agent")
    sub_graph.add_conditional_edges("agent", tools_condition)
    sub_graph.add_edge("tools", "agent")
    specialist_app = sub_graph.compile()

    def specialist_node(state: MultiAgentState):
        console.print(f"--- CALLING {output_key.replace('_report','').upper()} ANALYST ---")
        result = specialist_app.invoke(
            {
                "messages": [
                    SystemMessage(content=system_prompt),
                    HumanMessage(content=state["user_request"]),
                ]
            },
            config={"recursion_limit": 25},
        )
        content = get_last_ai_content(result["messages"])
        return {output_key: content}

    return specialist_node


# Create the specialist nodes with deep, domain-specific playbooks.
# This is the core advantage of multi-agent: each specialist receives
# far richer instructions than a single generalist ever could.

news_analyst_node = create_specialist_node(
    """You are a Senior News & Sentiment Analyst at a top-tier investment research firm. You deliver sharp, data-driven briefings on the current news landscape and market sentiment.

Your playbook:
1. Search for the LATEST news headlines, press releases, and analyst rating changes for the company.
2. Search for recent investor sentiment (e.g. "<company> stock sentiment" or "<company> analyst ratings").
3. In your report, include:
   - Specific headlines with their sources and approximate dates.
   - Any recent analyst upgrades, downgrades, or price target changes (include the firm name and target price).
   - An overall sentiment read: bullish, bearish, or mixed — and why.
   - Key catalysts or risks the market is currently focused on.

Format your output as a polished markdown section titled "## 1. News & Market Sentiment" with clear sub-points.""",
    "news_report"
)

technical_analyst_node = create_specialist_node(
    """You are a Senior Technical Analyst and Chartered Market Technician (CMT). You read price action and technical indicators to assess the short- and medium-term trajectory of a stock.

Your playbook:
1. Search for the company's current stock price, 52-week high/low, and recent price action.
2. Search for technical analysis or chart patterns (e.g. "<company> technical analysis" or "<company> stock chart indicators").
3. In your report, include:
   - Current price vs. key moving averages (50-day, 200-day) and what the crossover implies.
   - RSI reading and whether the stock is overbought or oversold.
   - Key support and resistance levels with specific price points.
   - Recent volume trends and what they signal.
   - A short-term technical outlook (bullish, bearish, or neutral) with justification.

Format your output as a polished markdown section titled "## 2. Technical Analysis" with specific numbers for every indicator.""",
    "technical_report"
)

financial_analyst_node = create_specialist_node(
    """You are a Senior Equity Research Analyst (CFA) specializing in fundamental analysis. You dissect financial statements to assess a company's intrinsic value and financial health.

Your playbook:
1. Search for the company's most recent quarterly earnings results (revenue, EPS, guidance).
2. Search for the company's key financial ratios and valuation metrics (P/E, P/S, margins, revenue growth YoY).
3. In your report, include:
   - Latest reported revenue and EPS vs. analyst consensus estimates (beat or miss, and by how much).
   - Revenue growth rate (YoY) and the trend over the last 2-3 quarters.
   - Gross margin, operating margin, and any notable changes.
   - Forward guidance from management and how it compares to Street expectations.
   - Current valuation (P/E, forward P/E) relative to sector peers.

Format your output as a polished markdown section titled "## 3. Financial Performance" with specific dollar amounts, percentages, and comparisons.""",
    "financial_report"
)

def report_writer_node(state: MultiAgentState):
    """The manager agent that synthesizes the specialist reports."""
    console.print("--- CALLING REPORT WRITER ---")
    prompt = f"""You are a Senior Financial Editor at a leading investment research firm. You have received three specialist reports from your analyst team. Combine them into a single, polished, publication-ready market analysis report.

Instructions:
- Begin with a brief "## Executive Summary" (2-3 sentences capturing the overall investment picture).
- Preserve the numbered section headings from each specialist (## 1., ## 2., ## 3.).
- Keep ALL specific data points, numbers, and metrics from the specialists — do not generalize or water them down.
- End with a brief "## Outlook" paragraph that synthesizes the three perspectives into a forward-looking view.
- The tone should be professional, confident, and concise — like a top-tier equity research note.

--- NEWS & SENTIMENT REPORT ---
{state['news_report']}

--- TECHNICAL ANALYSIS REPORT ---
{state['technical_report']}

--- FINANCIAL PERFORMANCE REPORT ---
{state['financial_report']}
"""
    final_report = llm.invoke(prompt).content
    return {"final_report": final_report}

print("Specialist agent nodes and Report Writer node defined.")

### Step 2.2: Building the Multi-Agent Graph

**What we are going to do:**
Now we'll wire the specialists and the manager into a graph. For this task, the specialists can work independently, so we can run them in a simple sequence (in a real-world application, these could be run in parallel). The final step is always the report writer.

In [ ]:
multi_agent_graph_builder = StateGraph(MultiAgentState)

# Add all the nodes
multi_agent_graph_builder.add_node("news_analyst", news_analyst_node)
multi_agent_graph_builder.add_node("technical_analyst", technical_analyst_node)
multi_agent_graph_builder.add_node("financial_analyst", financial_analyst_node)
multi_agent_graph_builder.add_node("report_writer", report_writer_node)

# Define the workflow sequence
multi_agent_graph_builder.set_entry_point("news_analyst")
multi_agent_graph_builder.add_edge("news_analyst", "technical_analyst")
multi_agent_graph_builder.add_edge("technical_analyst", "financial_analyst")
multi_agent_graph_builder.add_edge("financial_analyst", "report_writer")
multi_agent_graph_builder.add_edge("report_writer", END)

multi_agent_app = multi_agent_graph_builder.compile()
print("Multi-agent specialist team compiled successfully.")

## Phase 3: Head-to-Head Comparison

Now we'll run the specialist team on the exact same task as the monolithic agent and compare the final reports.

In [ ]:
multi_agent_query = monolithic_query
initial_multi_agent_input = {"user_request": multi_agent_query}

console.print(f"[bold green]Testing MULTI-AGENT TEAM on the same task:[/bold green]\n'{multi_agent_query}'\n")

final_multi_agent_output = multi_agent_app.invoke(initial_multi_agent_input)

console.print("\n--- [bold green]Final Report from Multi-Agent Team[/bold green] ---")
console.print(Markdown(final_multi_agent_output['final_report']))

**Discussion of the Output:**
The difference in the final report is significant. The output from the multi-agent team is:
- **Highly Structured:** It has clear, distinct sections for each area of analysis because each was generated by a specialist with a specific formatting instruction.
- **Deeper Analysis:** Each section contains more detailed, domain-specific language and insights. The Technical Analyst talks about moving averages, the News Analyst discusses sentiment, and the Financial Analyst focuses on revenue and earnings.
- **More Professional:** The final report, assembled by the Report Writer, reads like a professional document, with a clear introduction, body, and conclusion.

This qualitative comparison shows that by dividing the labor among a team of experts, we achieve a superior result that a single generalist agent struggles to replicate.

## Phase 4: Quantitative Evaluation

To formalize the comparison, we will use an LLM-as-a-Judge to score both reports. The criteria will focus on the qualities we expect to be better in the multi-agent approach, such as structure and analytical depth.

In [ ]:
class ComparativeEvaluation(BaseModel):
    """Side-by-side evaluation forces the judge to differentiate between reports,
    avoiding the score-compression problem of independent absolute scoring."""
    report_a_clarity_score: int = Field(
        description="Score 1-10 for Report A. High scores require professional section headings, logical flow, and a polished intro/conclusion."
    )
    report_a_depth_score: int = Field(
        description="Score 1-10 for Report A. High scores require specific numbers, concrete data points, and expert-level interpretation."
    )
    report_a_completeness_score: int = Field(
        description="Score 1-10 for Report A. High scores require substantive coverage of ALL three areas: news, technical, and financial."
    )
    report_b_clarity_score: int = Field(
        description="Score 1-10 for Report B. High scores require professional section headings, logical flow, and a polished intro/conclusion."
    )
    report_b_depth_score: int = Field(
        description="Score 1-10 for Report B. High scores require specific numbers, concrete data points, and expert-level interpretation."
    )
    report_b_completeness_score: int = Field(
        description="Score 1-10 for Report B. High scores require substantive coverage of ALL three areas: news, technical, and financial."
    )
    justification: str = Field(
        description="2-3 sentences explaining the key quality differences between the two reports."
    )

judge_llm = llm.with_structured_output(ComparativeEvaluation)

mono_report = get_last_ai_content(final_mono_output['messages'])
multi_report = final_multi_agent_output['final_report']

prompt = f"""You are a strict, expert financial report judge. You are given TWO reports written in response to the SAME user request. Read both carefully, then score each one independently.

IMPORTANT RULES:
- Scores MUST reflect real quality differences between the reports.
- Count specific data points in each report: exact stock prices, percentage changes, named analyst firms, specific financial metrics (EPS, P/E ratios, margin percentages, revenue figures). The report with MORE concrete data deserves a higher depth score.
- Evaluate structure: the report with clearer section headings, an executive summary, and a professional conclusion deserves a higher clarity score.
- Do NOT give both reports the same score on any criterion unless they are genuinely indistinguishable on that dimension.
- Scoring guide: 5 = average, 7 = good, 9-10 = exceptional/publication-ready.

**Original User Request:**
{monolithic_query}

**Report A:**
{mono_report}

**Report B:**
{multi_report}
"""

console.print("--- Comparative Evaluation (both reports judged side-by-side) ---\n")
result = judge_llm.invoke(prompt)

mono_scores = {
    "clarity_and_structure": result.report_a_clarity_score,
    "analytical_depth": result.report_a_depth_score,
    "completeness": result.report_a_completeness_score,
}
multi_scores = {
    "clarity_and_structure": result.report_b_clarity_score,
    "analytical_depth": result.report_b_depth_score,
    "completeness": result.report_b_completeness_score,
}

console.print("[bold yellow]Monolithic Agent (Report A):[/bold yellow]")
console.print(mono_scores)
console.print("\n[bold green]Multi-Agent Team (Report B):[/bold green]")
console.print(multi_scores)
console.print(f"\n[bold]Justification:[/bold] {result.justification}")

**Discussion of the Output:**

Notice that we use a **side-by-side comparative evaluation** rather than scoring each report independently. This is a deliberate methodological choice: when LLMs score reports in isolation, they tend to compress scores into a narrow band (e.g. everything gets 8s and 9s), making it hard to distinguish quality differences. By showing the judge both reports at once, it is forced to directly compare them — counting data points, comparing structure, and identifying which report provides deeper analysis.

The **Multi-Agent Team** should score meaningfully higher, especially on `analytical_depth`. Each specialist was armed with a domain-specific "playbook" telling it exactly what to search for and what metrics to report, producing sections dense with concrete data. The monolithic agent, working from a single generic prompt, cannot match this level of targeted depth.

This evaluation confirms that for complex tasks that can be decomposed into domains of expertise, the Multi-Agent architecture — combined with targeted prompt engineering per specialist — is a superior approach for generating high-quality, structured, and reliable results.

## Conclusion

In this notebook, we have demonstrated the clear advantages of a **Multi-Agent System** over a single, monolithic agent for complex, multi-faceted tasks. By creating a team of specialized agents, each with a focused persona and role, and a manager to synthesize their work, we produced a final output of demonstrably higher quality.

The key takeaway is the power of **specialization**. Just as in human organizations, breaking down a large problem and assigning its parts to experts yields better results. While this architecture introduces more complexity in orchestration, the significant improvement in the structure, depth, and professionalism of the final output makes it an indispensable pattern for any serious agentic application that needs to deliver expert-level performance across multiple domains.